In [17]:
import pandas as pd
import ast
import asyncio
import os
from typing import List, Dict
from tqdm.asyncio import tqdm_asyncio
import json
import random
from queue import deque

# WEBGLM

In [4]:
with open('webglm.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

In [12]:
with open('test/webglm_test.jsonl', 'w') as f:
    for item in data[:1000]:
        json.dump(item, f)
        f.write('\n')

In [13]:
with open('val/webglm_val.jsonl', 'w') as f:
    for item in data[1000:4100]:
        json.dump(item, f)
        f.write('\n')

In [14]:
with open('train/webglm_train.jsonl', 'w') as f:
    for item in data[4100:]:
        json.dump(item, f)
        f.write('\n')

# RAFT

In [3]:
data = []

with open('webglm_raft.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

In [4]:
print(len(data))

48468


In [5]:
with open('val/webglm_raft_val.jsonl', 'w') as f:
    for item in data[:1000]:
        json.dump(item, f)
        f.write('\n')
    for item in data[-100:]:
        json.dump(item, f)
        f.write('\n')

In [ ]:
# with open('val/webglm_raft_val.jsonl', 'w') as f:
#     for item in data[1000:4000]:
#         json.dump(item, f)
#         f.write('\n')
#     for item in data[-400:-100]:
#         json.dump(item, f)
#         f.write('\n')

In [6]:
with open('train/webglm_raft_train.jsonl', 'w') as f:
    for item in data[1000:-100]:
        json.dump(item, f)
        f.write('\n')

# Lost In The Middle

In [18]:
df = pd.read_excel('webglm_full.xlsx')

def create_llama_dataset(df: pd.DataFrame) -> List[Dict]:
    dataset = []
    for _, row in df.iterrows():
        question = row['question']
        answer = str(row['answer'])
        examples = ast.literal_eval(row["references"]) if isinstance(row['references'], str) else []
        
        dataset_item = {
            "question": question,
            "answer": answer,
            "references": examples
        }
        dataset.append(dataset_item)
    return dataset


dataset = create_llama_dataset(df)[:100]

max_refs = 9
storage = deque(maxlen=max_refs)

for item in dataset:
    storage.append(item)

In [19]:
SYSTEM_PROMPT = (
    "Ты — экспертная система Compressa RAG, "
    "предоставляющая точные и релевантные ответы на вопросы, "
    "используя только предоставленную контекстную информацию. "
    "Отвечай на русском языке."
)
REJECT_ANSW = "К сожалению, ответа на вопрос нет в упомянутых источниках"

def get_summary_prompt(context_list, question):
    context = ''
    for i, c in enumerate(context_list):
        context += f'Источник [{i+1}]:'+"\n"+c+"\n\n"

    prompt = (
        f"# Контекстная информация:\n\n{context}\n\n"
        "---\n"
        "# Инструкции:\n\n"
        "1. Дай полный и краткий ответ на вопрос, используя только информацию из контекста.\n"
        "2. Указывай источники, с которыми работаешь, например: [1].\n"
        "3. Ответ должен содержать хотя бы одну ссылку на источник.\n"
        f"4. Ответ гарантированно содержится в указанных источниках.\n"
        "5. Не используй знания вне предоставленного контекста.\n\n"
        f"# Вопрос:\n\n{question}\n\n"
    )
    return prompt

def update_refs_number(answer, old_numbers, new_numbers):
    for old, new in zip(old_numbers, new_numbers):
        answer = answer.replace(f"[{old}]", f"[{new}]")
    return answer

In [20]:
with open("test/lost_in_the_middle.jsonl", "w") as output:
    for idx, item in enumerate(dataset[:10]):
        
        true_refs = item["references"]

        for golden_position in range(0, 10, 1):
            all_refs = []
            shift = 0
            for j in range(10):
                if golden_position == j:
                    all_refs += true_refs
                    shift = 1
                else:
                    all_refs += storage[j-shift]["references"]

            new_numbers = [all_refs.index(ref) + 1 for ref in true_refs]
            answer = update_refs_number(item["answer"], list(range(1, len(new_numbers)+1)), new_numbers)
        
            data = ({
                "messages": [
                    {"content": SYSTEM_PROMPT, "role": "system"},
                    {"content": get_summary_prompt(all_refs, item["question"]), "role": "user"},
                ],
                "golden_answer": answer,
                "question_idx": idx,
                "golden_position": golden_position,
            })

            json.dump(data, output)
            output.write('\n')

        storage.append(item)

In [21]:
with open('test/lost_in_the_middle.jsonl', 'r') as f:
    print(len(f.readlines()))

100


In [22]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
from transformers import AutoModelForCausalLM
import json
import torch

tokens_count = 0
arr = []

with open('test/lost_in_the_middle.jsonl', 'r') as f:
    dataset = f.readlines()

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B")

for row in dataset:
    item = json.loads(row)
    # print(item['messages'][1])
    # break
    l = len(tokenizer.encode(item['messages'][1]['content'], add_special_tokens=False))
    tokens_count += l
    arr.append(l)

In [23]:
import numpy as np

arr = np.sort(np.array(arr))

print(f"question_avg {tokens_count / len(dataset)}")
print(f"question_max {arr[-1]}")
print(f"question_80 {arr[int(len(dataset)*0.8)]}")
print(f"question_95 {arr[int(len(dataset)*0.95)]}")
print(f"question_99 {arr[int(len(dataset)*0.99)]}")

question_avg 7005.0
question_max 7803
question_80 7541
question_95 7803
question_99 7803


In [24]:
import json

with open('test/lost_in_the_middle.jsonl', 'r') as f:
    item = json.loads(f.readlines()[1])
    print(item['messages'][1]['content'])

    print(item['golden_answer'])

# Контекстная информация:

Источник [1]:
Eli5

Источник [2]:
Мы не "теряем" пресную воду в прямом смысле. Мы просто используем ограниченный запас, который у нас есть, с большей скоростью, чем природа его пополняет.

Источник [3]:
Ответ заключается в том, что людям для выживания необходим доступ к пресной жидкой воде, которая является лишь небольшой частью общего водного цикла. Когда городские чиновники говорят вам не тратить воду напрасно, они имеют в виду, что вы не должны бездумно перенаправлять воду из части водного цикла, доступной для использования человеком, в часть, не доступную для использования человеком. Согласно книге "Вода в кризисе", под редакцией Питера Г. Глейка, только 0,77% воды на Земле находится в виде пресной, жидкой воды на поверхности земли и в земле. Остальная вода находится в соленых океанах и озерах, заключена в льду или в атмосфере. Позволять воде стекать в раковину, пока вы берете перекус, обычно означает, что эта вода выходит из части водного цикла, доступно